# Web Scraping User Ratings From AlloCiné.fr

This script builds a DataFrame by web scraping the data from AlloCiné — this time, its purpose is to retrieve the user and press ratings of movies and series from the website of AlloCiné. We will proceed as follows: 
- We use the series and movies url lists generated in the other scripts to get the comments section urls of the press and the spectators (users) with `getCommentsUrl()`.
- We use the press comments urls for movies and series to get the press ratings with `getPressRatings()`.
- We use the user comments urls for movies and series to get the user ratings with `getUserRatings()`.
- From there, we scrape the user ID and their rating for each movie or series with `ScrapeURL()`.
- The user can be either a person or a press newspaper. Separated files are generated for each type of user.

*Note : We use the popular BeautifulSoup package*

## Functions :

### `getCommentsUrl(movies_df, series_df, spect, press)`

This function will call two sub-functions: `getMoviesCommentsUrl(movies_df, spect, press)` and `getSeriesCommentsUrl(series_df, spect, press)`. Both respectively iterate over the list of movies and series url generated by `getMoviesUrl()` in the previous scripts and get the comments section url. We can chose to get the comments section from the user or the press for each video type.
We then store the lists of urls in a csv file entitled `user_comments_url.csv` (resp. `press_comments_url.csv`), in both movies and series directory (`../Movies/Comments/` and `../Series/Comments/`).

### `ScrapeURL(urls)` :

Iterate over the list of movies or series comments section url generated by `getCommentsUrl()` and scrape the data for each movie or series ratings. In the process, we extract :

- `user_id` : Allocine user id (person or press)
- `id` : Allocine movie or series id
- `user_rating`: AlloCiné users ratings (from 0.5 to 5 stars) 


The function `ScrapeURL()` returns two objects : the user_rating and press-rating as two dataframes. In addition, the two objects are saved as `user_ratings_movies.csv` and `press_ratings_movies.csv` (respectively `user_ratings_series.csv` and `press_ratings_series.csv`).

-------------------------------------------------------------------
## Import libs

In [1]:
# Import libs
import pandas as pd
import numpy as np
from time import time
from time import sleep
import re
from datetime import timedelta
from urllib.request import urlopen
from random import randint
from bs4 import BeautifulSoup
import os
import dateparser

from warnings import warn
from IPython.core.display import clear_output
import traceback

C:\Users\basti\AppData\Local\Temp\ipykernel_29672\1268171875.py:15: DeprecationWarning: Importing clear_output from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import clear_output


## Functions

In [2]:
# A SUPPRIMER APRES TESTS
response = urlopen("https://www.allocine.fr/film/fichefilm-211012/critiques/spectateurs/")
html_text = response.read().decode("utf-8")
spect_html_soup = BeautifulSoup(html_text, 'html.parser')

In [3]:
int(spect_html_soup.find_all('span', {'class': 'button-md'})[-1].text)

89

### Convert French months to English months: `convert_month(month)`

In [4]:
# Convert french months to english months
def convert_month(month):
    if month == "janvier":
        return "January"
    elif month == "février":
        return "February"
    elif month == "mars":
        return "March"
    elif month == "avril":
        return "April"
    elif month == "mai":
        return "May"
    elif month == "juin":
        return "June"
    elif month == "juillet":
        return "July"
    elif month == "août":
        return "August"
    elif month == "septembre":
        return "September"
    elif month == "octobre":
        return "October"
    elif month == "novembre":
        return "November"
    elif month == "décembre":
        return "December"
    else:
        return "Unknown"


### Function: `getCommentsUrl(movies_df, series_df, spect, press)`

#### Function: `getMoviesCommentsUrl(movies_df, spect, press)`

In [5]:
def getMoviesCommentsUrl(movies_df: pd.DataFrame, spect: bool=False, press: bool=False):
    '''
    Get the comments section url for each movie
    You can select the spectateurs or presse section, or both.
    :param movies_df: DataFrame of movies
    :param spect: Boolean, True if you want to get the spectateurs section
    :param press: Boolean, True if you want to get the presse section
    :return: nothing but saves urls in csv files
    '''    
    # Get the list of movies_id from the movies_df
    movies_id_list = movies_df['id'].tolist()

    # Set the list
    press_url_list = []
    user_url_list = []

    # Preparing the setting and monitoring of the loop
    start_time = time()
    p_requests = 1
        
    for v_id in movies_id_list:
        if spect:
            # Url for the spectators/user section sorted by the descencing number of reviews per user
            url_spect = f'https://www.allocine.fr/film/fichefilm-{v_id}/critiques/spectateurs/membres-critiques/'    
            user_url_list.append(url_spect)
        if press:
            url_press = f'https://www.allocine.fr/film/fichefilm-{v_id}/critiques/presse/'   
            press_url_list.append(url_press)                         
        p_requests += 1

    # Saving the files
    comments_path = '../Movies/Comments/'
    os.makedirs(os.path.dirname(comments_path), exist_ok=True) #create folders if not exists
    print(f'--> Done; {p_requests-1} Movies Comments Page Requests in {timedelta(seconds=time()-start_time)}')
    if spect:
        r = np.asarray(user_url_list)
        np.savetxt(f"{comments_path}user_comments_urls.csv", r, delimiter=",", fmt='%s')
    if press:
        r = np.asarray(press_url_list)
        np.savetxt(f"{comments_path}press_comments_urls.csv", r, delimiter=",", fmt='%s')

#### Function: `getSeriesCommentsUrl(series_df, spect, press)`

In [6]:
def getSeriesCommentsUrl(series_df: pd.DataFrame, spect: bool=False, press: bool=False):
    '''
    Get the comments section url for each series
    You can select the spectateurs or presse section, or both.
    :param series_df: DataFrame of series
    :param spect: Boolean, True if you want to get the spectateurs section
    :param press: Boolean, True if you want to get the presse section
    :return: nothing but saves urls in  csv files
    '''
    # Get the list of series_id from the series_df
    series_id_list = series_df['id'].tolist()

    # Set the list
    press_url_list = []
    user_url_list = []

    # Preparing the setting and monitoring of the loop
    start_time = time()
    p_requests = 1
        
    for v_id in series_id_list:
        if spect:
            # Url for the spectators/user section sorted by the descencing number of reviews per user
            url_spect = f'https://www.allocine.fr/series/ficheserie-{v_id}/critiques/membres-critiques/'    
            user_url_list.append(url_spect)
        if press:
            url_press = f'https://www.allocine.fr/series/ficheserie-{v_id}/critiques/presse/'   
            press_url_list.append(url_press)                         
        p_requests += 1

    # Saving the files
    comments_path = '../Series/Comments/'
    os.makedirs(os.path.dirname(comments_path), exist_ok=True) #create folders if not exists
    print(f'--> Done; {p_requests-1} Series Comments Page Requests in {timedelta(seconds=time()-start_time)}')
    if spect:
        r = np.asarray(user_url_list)
        np.savetxt(f"{comments_path}user_comments_urls.csv", r, delimiter=",", fmt='%s')
    if press:
        r = np.asarray(press_url_list)
        np.savetxt(f"{comments_path}press_comments_urls.csv", r, delimiter=",", fmt='%s')

#### Main function: `getCommentsUrl(movies_df, series_df, spect, press)`

In [7]:
def getCommentsUrl(movies_df=None, series_df=None, spect=False, press=False):
    '''
    Get the comments section url for each movie
    You can select the spectateurs or presse section, or both.
    :param movies_df: DataFrame of movies
    :param series_df: DataFrame of series
    :param spect: Boolean, True if you want to get the spectateurs section
    :param press: Boolean, True if you want to get the presse section
    :return: nothing but saves urls in a csv file
    '''
    try:
        if movies_df is not None:
            getMoviesCommentsUrl(movies_df, spect, press)
        if series_df is not None:
            getSeriesCommentsUrl(series_df, spect, press)
    except:
        print('Error in getCommentsUrl function!')
        traceback.print_exc()

### Get press-ratings dataframe: `getPressRatings(urls)`

#### Convert text to rating: `convertTextToRating(text)`

In [8]:
def convert_text_to_rating(text):
    '''
    Convert text to rating
    :param text: Evaluation of the movie as a string 
    :return: corresponding float rating value
    '''
    if text=="Nul":
        return 0.5
    elif text=="Très mauvais":
        return 1
    elif text=="Mauvais":
        return 1.5
    elif text=="Pas terrible":
        return 2
    elif text=="Moyen":
        return 2.5  
    elif text=="Pas mal":
        return 3
    elif text=="Bien":
        return 3.5
    elif text=="Très bien":
        return 4
    elif text=="Excellent":
        return 4.5
    elif text=="Chef d'oeuvre":
        return 5

#### Main function: `getPressRatings(urls)`

In [9]:
def getPressRatings(press_soup):
    '''
    Get press_ratings from the comments section url for each movie or series.
    :param press_soup: BeautifulSoup object of the press comments section url
    :return: a dictionary of ratings (key: user_id, value: press rating) if the ratings are available, else None.
    '''
    press_ratings = {}
    div_ratings = press_soup.find_all('li', {'class': 'item'})
    if div_ratings:
        press_ratings = {id.text.strip():convert_text_to_rating(rating.find('span')['title']) for id, rating in zip(div_ratings, div_ratings)}
        return press_ratings
    return None
    

### Get user-ratings dataframe: `getUserRatings(urls)`

In [10]:
def getUserRatings(user_rating_url, last_page: int=1, nb_users: int=5):
    '''
    Get user_ratings from the comments section url for each movie or series.
    We will keep only the nb_users first users who have posted the most reviews.
    :param user_rating_url : url of the user ratings section.
    :param last_page: last page of the user ratings section (default 1).
    :param nb_users: Number of users to keep (default 5).
    :return: a dictionary of ratings {key=(user_id, user_name): value=(user rating, date)} if the ratings are available, else None.
    '''
    user_ratings = {}
    for n_page in range(1,last_page+1):
        # If the number of users is reached, stop the loop.
        # Else, get the next page of the user ratings section until the last page is reached.
        if len(user_ratings) == nb_users:       
            break

        user_rating_url_page = user_rating_url + f'?page={n_page}'        
        try :
            response = urlopen(user_rating_url_page)
            # Pause the loop
            sleep(randint(1,2))
            # Parse the content of the request with BeautifulSoup
            html_text = response.read().decode("utf-8")
            user_soup = BeautifulSoup(html_text, 'html.parser')
            div_ratings = user_soup.find_all('div', {'class': 'review-card'})
            if div_ratings:
                for item in div_ratings:
                    if len(user_ratings) == nb_users:       
                        break      
                    user_id = item.find('span', {'class': 'item-profil'})['data-targetuserid']
                    user_name = item.span['title']
                    user_rating = float(re.sub(",", ".", item.find("span", {"class": "stareval-note"}).text))
                    user_date = item.find('span', {'class': 'review-card-meta-date'}).text.strip().replace('Publiée le ', '')
                    month = user_date.split(' ')[1]
                    user_date = dateparser.parse(user_date.replace(month, convert_month(month)), date_formats=["%d %B %Y"]).date()
                    user_ratings[(user_id, user_name)] = (user_rating, user_date)
        except:
            print(f'Error in getUserRatings function!')
            traceback.print_exc()
    return user_ratings

### Function: `ScrapeURL(series_urls, movies_urls, press, spect)`

#### Function: `ScrapePressURL(series_urls, movies_urls)`

In [11]:
def ScrapePressURL(series_url: list=None, movies_url: list=None):   
    '''
    Scrape the press_ratings from the series and/or movies press comments section urls (if available).
    :param series_url: list of series press comments section urls.
    :param movies_url: list of movies press comments section urls.
    :return: dataframes of series and movies press ratings and the errors.
    '''
    # init the dataframes    
    if series_url is None and movies_url is None:
        print('No url to scrape!')
        return None, None, None, None
    if series_url is not None:
        c = ["press_id",
        "series_id",
        "press_rating",]
        df_series = pd.DataFrame(columns=c)
    if movies_url is not None:
        c = ["press_id",
        "movie_id",
        "press_rating",]
        df_movies = pd.DataFrame(columns=c)
    
    # preparing the setting and monitoring loop
    start_time = time()
    ns_request = 0
    nm_request = 0
    
    # init list to save errors
    errors_series = []
    errors_movies = []
    
    if series_url is not None:
        # request loop
        for url in series_url:
            try :
                response = urlopen(url)

                # Pause the loop
                sleep(randint(1,2))

                # Monitoring the requests
                ns_request += 1
                
                elapsed_time = time() - start_time
                print(f'Series Press Request #{ns_request}; Frequency: {ns_request/elapsed_time} requests/s')
                clear_output(wait = True)

                # Pause the loop
                sleep(randint(1,2))

                # Warning for non-200 status codes
                if response.status != 200:
                    warn('Series Press Request #{}; Status code: {}'.format(ns_request, response.status_code))
                    errors_series.append(url)

                # Parse the content of the request with BeautifulSoup
                html_text = response.read().decode("utf-8")
                press_html_soup = BeautifulSoup(html_text, 'html.parser')

                # Get the series_id
                series_id = url.split('/')[-4].split('-')[-1]            
                # Get the press_ratings
                press_ratings = getPressRatings(press_html_soup)
                for id, rating in press_ratings.items():               
                    # Append the data
                    df_series_tmp = pd.DataFrame({'press_id': [id],
                                        'series_id': [series_id],
                                        'press_rating': [rating]
                                        })                                    
                    df_series = pd.concat([df_series, df_series_tmp], ignore_index=True)                
            except:
                errors_series.append(url)
                warn(f'Series Press Request #{ns_request} fail; Press rating does not exist! Total errors : {len(errors_series)}')
                traceback.print_exc()
    if movies_url is not None:
        # request loop
        for url in movies_url:
            try :
                response = urlopen(url)

                # Pause the loop
                sleep(randint(1,2))

                # Monitoring the requests
                nm_request += 1
                
                elapsed_time = time() - start_time
                print(f'Movie Press Request #{nm_request}; Frequency: {nm_request/elapsed_time} requests/s')
                clear_output(wait = True)

                # Pause the loop
                sleep(randint(1,2))

                # Warning for non-200 status codes
                if response.status != 200:
                    warn('Movie Press Request #{}; Status code: {}'.format(nm_request, response.status_code))
                    errors_movies.append(url)

                # Parse the content of the request with BeautifulSoup
                html_text = response.read().decode("utf-8")
                press_html_soup = BeautifulSoup(html_text, 'html.parser')

                # Get the movie_id
                movie_id = url.split('/')[-4].split('-')[-1]            
                # Get the press_ratings
                press_ratings = getPressRatings(press_html_soup)
                for id, rating in press_ratings.items():               
                    # Append the data
                    df_movies_tmp = pd.DataFrame({'press_id': [id],
                                        'movie_id': [movie_id],
                                        'press_rating': [rating]
                                        })                                    
                    df_movies = pd.concat([df_movies, df_movies_tmp], ignore_index=True)                
            except:
                errors_movies.append(url)
                warn(f'Movie Press Request #{nm_request} fail; Press rating does not exist! Total errors : {len(errors_movies)}')
                traceback.print_exc()
            
    # monitoring 
    if series_url is not None:
        series_path = '../Series/Ratings/'
        os.makedirs(os.path.dirname(series_path), exist_ok=True) #create folders if not exists
        elapsed_time = time() - start_time
        print(f'Done; {ns_request} Series Press Ratings requests in {timedelta(seconds=elapsed_time)} with {len(errors_series)} errors (series with no press ratings)')
        df_series.to_csv(f"{series_path}press_ratings_series.csv", index=False)
        # list to dataframe
        errors_df_series = pd.DataFrame(errors_series, columns=['url'])
        errors_df_series.to_csv(f"{series_path}press_ratings_errors.csv")
    if movies_url is not None:
        movies_path = '../Movies/Ratings/'
        os.makedirs(os.path.dirname(movies_path), exist_ok=True) #create folders if not exists
        elapsed_time = time() - start_time
        print(f'Done; {nm_request} Movies Press Ratings requests in {timedelta(seconds=elapsed_time)} with {len(errors_movies)} errors (movies with no press ratings)')
        df_movies.to_csv(f"{movies_path}press_ratings_movies.csv", index=False)
        # list to dataframe
        errors_df_movies = pd.DataFrame(errors_movies, columns=['url'])
        errors_df_movies.to_csv(f"{movies_path}press_ratings_errors.csv")
    
    # return dataframe and errors
    return df_series, df_movies, errors_series, errors_movies

#### Function: `ScrapeUserURL(series_urls, movies_urls)`

In [12]:
def ScrapeUserURL(series_url: list=None, movies_url: list=None):
    '''
    Scrape the user_ratings from the series and/or movies press comments section urls (if available).
    :param series_url: list of series user comments urls
    :param movies_url: list of movies user comments urls
    :return: dataframes of series and movies user ratings and the errors.  
    '''
    # init the dataframes    
    if series_url is None and movies_url is None:
        print('No url to scrape!')
        return None, None, None, None
    if series_url is not None:
        c = ["user_id",
        "user_name",
        "series_id",
        "user_rating",
        "date",
    ]
        df_series = pd.DataFrame(columns=c)
    if movies_url is not None:
        c = ["user_id",
        "user_name",
        "movie_id",
        "user_rating",
        "date",
    ]
        df_movies = pd.DataFrame(columns=c)
    
    # preparing the setting and monitoring loop
    start_time = time()
    ns_request = 0
    nm_request = 0
    
    # init list to save errors
    errors_series = []
    errors_movies = []
    
    if series_url is not None:
        # request loop
        for url in series_url:
            try :
                response = urlopen(url)

                # Pause the loop
                sleep(randint(1,2))

                # Monitoring the requests
                ns_request += 1
                
                elapsed_time = time() - start_time
                print(f'Series User Request #{ns_request}; Frequency: {ns_request/elapsed_time} requests/s')
                clear_output(wait = True)

                # Pause the loop
                sleep(randint(1,2))

                # Warning for non-200 status codes
                if response.status != 200:
                    warn('Series User Request #{}; Status code: {}'.format(ns_request, response.status_code))
                    errors_series.append(url)

                # Parse the content of the request with BeautifulSoup
                html_text = response.read().decode("utf-8")
                user_html_soup = BeautifulSoup(html_text, 'html.parser')

                # Get the series_id
                series_id = url.split('/')[-4].split('-')[-1] 
                # Get the url last page
                last_page = int(user_html_soup.find_all('span', {'class': 'button-md'})[-1].text)
                # Get the user_ratings
                user_ratings = getUserRatings(url,last_page,nb_users=10)
                
                for id, rating in user_ratings.items():               
                    # Append the data
                    df_series_tmp = pd.DataFrame({'user_id': [id[0]],
                                        'user_name': [id[1]],
                                        'series_id': [series_id],
                                        'user_rating': [rating[0]],
                                        'date': [rating[1]]
                                        })                                    
                    df_series = pd.concat([df_series, df_series_tmp], ignore_index=True)                
            except:
                errors_series.append(url)
                warn(f'Series User Request #{ns_request} fail; User rating does not exist! Total errors : {len(errors_series)}')
                traceback.print_exc()

    if movies_url is not None:
        # request loop
        for url in movies_url:
            try :
                response = urlopen(url)

                # Pause the loop
                sleep(randint(1,2))

                # Monitoring the requests
                nm_request += 1
                
                elapsed_time = time() - start_time
                print(f'Movie User Request #{nm_request}; Frequency: {nm_request/elapsed_time} requests/s')
                clear_output(wait = True)

                # Pause the loop
                sleep(randint(1,2))

                # Warning for non-200 status codes
                if response.status != 200:
                    warn('Movie User Request #{}; Status code: {}'.format(nm_request, response.status_code))
                    errors_movies.append(url)

                # Parse the content of the request with BeautifulSoup
                html_text = response.read().decode("utf-8")
                user_html_soup = BeautifulSoup(html_text, 'html.parser')

                # Get the movie_id
                movie_id = url.split('/')[-5].split('-')[-1]            
                # Get the url last page
                last_page = int(user_html_soup.find_all('span', {'class': 'button-md'})[-1].text)
                # Get the user_ratings
                user_ratings = getUserRatings(url,last_page,nb_users=10)
                
                for id, rating in user_ratings.items():               
                    # Append the data
                    df_movies_tmp = pd.DataFrame({'user_id': [id[0]],
                                        'user_name': [id[1]],
                                        'movie_id': [movie_id],
                                        'user_rating': [rating[0]],
                                        'date': [rating[1]]
                                        })                                    
                    df_movies = pd.concat([df_movies, df_movies_tmp], ignore_index=True)                             
            except:
                errors_movies.append(url)
                warn(f'Movie User Request #{nm_request} fail; User rating does not exist! Total errors : {len(errors_movies)}')
                traceback.print_exc()
            
    # monitoring 
    if series_url is not None:
        series_path = '../Series/Ratings/'
        os.makedirs(os.path.dirname(series_path), exist_ok=True) #create folders if not exists
        elapsed_time = time() - start_time
        print(f'Done; {ns_request} Series User Ratings requests in {timedelta(seconds=elapsed_time)} with {len(errors_series)} errors (series with no user ratings)')
        df_series.to_csv(f"{series_path}user_ratings_series.csv", index=False)
        # list to dataframe
        errors_df_series = pd.DataFrame(errors_series, columns=['url'])
        errors_df_series.to_csv(f"{series_path}user_ratings_errors.csv")
    if movies_url is not None:
        movies_path = '../Movies/Ratings/'
        os.makedirs(os.path.dirname(movies_path), exist_ok=True) #create folders if not exists
        elapsed_time = time() - start_time
        print(f'Done; {nm_request} Movies User Ratings requests in {timedelta(seconds=elapsed_time)} with {len(errors_movies)} errors (movies with no user ratings)')
        df_movies.to_csv(f"{movies_path}user_ratings_movies.csv", index=False)
        # list to dataframe
        errors_df_movies = pd.DataFrame(errors_movies, columns=['url'])
        errors_df_movies.to_csv(f"{movies_path}user_ratings_errors.csv")
    
    # return dataframe and errors
    return df_series, df_movies, errors_series, errors_movies

#### Main function: `ScrapeURL(press_series_urls, press_movies_urls, user_series_urls, user_movies_urls)`

In [13]:
def ScrapeURL(press_series_urls: list=None, press_movies_urls: list=None, user_series_urls: list=None, user_movies_urls: list=None):
    '''
    Scrape the user and press ratings from the movie and series pages.
    :param press_series_urls: list of press series urls
    :param press_movies_urls: list of press movies urls
    :param user_series_urls: list of user series urls
    :param user_movies_urls: list of user movies urls
    :return: list of dataframes of user and press ratings and errors
    '''
    ratings,errors = [],[]  
    print('Scraping Press Ratings...')
    press_series_ratings, press_movies_ratings, press_errors_series, press_errors_movies = ScrapePressURL(press_series_urls, press_movies_urls)
    ratings.append(press_series_ratings)
    ratings.append(press_movies_ratings)
    errors.append(press_errors_series)
    errors.append(press_errors_movies)

    print('Scraping Spectator Ratings...')
    user_series_ratings, user_movies_ratings, user_errors_series, user_errors_movies = ScrapeUserURL(user_series_urls, user_movies_urls)
    ratings.append(user_series_ratings)
    ratings.append(user_movies_ratings)
    errors.append(user_errors_series)
    errors.append(user_errors_movies)
    return ratings, errors

## Loading the movies and series dataframes

In [14]:
# Load the movies and series dataframes
def loadDataFrames():
    '''
    Load the movies and series dataframes
    :return: movies and series dataframes
    '''
    movies_df = pd.read_csv('../Movies/Data/allocine_movies.csv', nrows=15)
    series_df = pd.read_csv('../Series/Data/allocine_series.csv', nrows=15)
    return movies_df, series_df
movies_df, series_df = loadDataFrames()

In [15]:
poster_links = {m_name:p_link for m_name, p_link in zip(movies_df["title"], movies_df["poster_link"])}
poster_links

{'En corps': 'https://fr.web.img4.acsta.net/c_310_420/pictures/22/02/09/15/22/3063490.jpg',
 'The Batman': 'https://fr.web.img3.acsta.net/c_310_420/pictures/22/02/16/17/42/3125788.jpg',
 'Goliath': 'https://fr.web.img3.acsta.net/c_310_420/pictures/22/01/24/12/13/4677076.jpg',
 'Notre-Dame brûle': 'https://fr.web.img6.acsta.net/c_310_420/pictures/22/02/11/12/58/0195580.jpg',
 'Morbius': 'https://fr.web.img5.acsta.net/c_310_420/pictures/22/03/28/09/03/5612671.jpg',
 "Qu'est-ce qu'on a tous fait au Bon Dieu ?": 'https://fr.web.img5.acsta.net/c_310_420/pictures/21/10/21/10/17/5465951.jpg',
 'Permis de construire': 'https://fr.web.img6.acsta.net/c_310_420/pictures/21/10/18/09/57/1868139.jpg',
 'Le Temps des secrets': 'https://fr.web.img3.acsta.net/c_310_420/pictures/21/12/23/08/44/0309697.jpg',
 'Sonic 2 le film': 'https://fr.web.img3.acsta.net/c_310_420/pictures/22/03/14/15/39/4137538.jpg',
 'Ambulance': 'https://fr.web.img6.acsta.net/c_310_420/pictures/22/02/21/18/59/0222383.jpg',
 'À ple

## Getting the comments section urls from spectators and press for movies and series

In [15]:
getCommentsUrl(movies_df=movies_df, series_df=series_df, spect=True, press=True)

--> Done; 15 Movies Comments Page Requests in 0:00:00.001034
--> Done; 15 Series Comments Page Requests in 0:00:00.001015


## Loading the comments section urls

### For Series

#### From the press

In [16]:
# Getting the press_comments_urls for series
press_series_url = pd.read_csv("../Series/Comments/press_comments_urls.csv",names=['url'])
press_series_url = press_series_url['url'].tolist()

#### From the users

In [17]:
# Getting the user_comments_urls for series
user_series_url = pd.read_csv("../Series/Comments/user_comments_urls.csv",names=['url'])
user_series_url = user_series_url['url'].tolist()

### For Movies

#### From the press

In [18]:
# Getting the press_comments_urls for movies
press_movies_url = pd.read_csv("../Movies/Comments/press_comments_urls.csv",names=['url'])
press_movies_url = press_movies_url['url'].tolist()


#### From the users

In [19]:
# Getting the users_comments_urls for movies
user_movies_url = pd.read_csv("../Movies/Comments/user_comments_urls.csv",names=['url'])
user_movies_url = user_movies_url['url'].tolist()

## Scraping the data

In [36]:
# Scraping the user and/or press ratings from series and/or movies
ratings, errors = ScrapeURL(press_series_urls=None, press_movies_urls=None, user_series_urls=user_series_url, user_movies_urls=user_movies_url)

Done; 15 Series User Ratings requests in 0:04:25.685541 with 1 errors (series with no user ratings)
Done; 15 Movies User Ratings requests in 0:04:25.691344 with 0 errors (movies with no user ratings)


In [44]:
press_series_ratings = ratings[0]
press_movies_ratings = ratings[1]
user_series_ratings = ratings[2]
user_movies_ratings = ratings[3]

In [43]:
user_series_ratings.user_id.nunique()

847

In [102]:
errors

[None, None, None, None]

In [33]:
# Sort df_press_ratings by user_id
# press_series_ratings = press_series_ratings.sort_values(by=['user_id'],ignore_index=True)
# press_series_ratings.user_id.value_counts()

Variety                    75
Télérama                   74
The Hollywood Reporter     57
Le Parisien                57
Le Monde                   51
                           ..
Season One                  1
Nice-Matin                  1
Philadelphia Daily News     1
Red Eye                     1
The Salt Lake Tribune       1
Name: user_id, Length: 120, dtype: int64